In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!cp "/content/drive/MyDrive/PI2/dataset_120_real.zip" .
!cp "/content/drive/MyDrive/PI2/dataset_120.zip" .


In [6]:
import zipfile

with zipfile.ZipFile("dataset_120_real.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset_120_real")

with zipfile.ZipFile("dataset_120.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset_120")


In [50]:
!mv dataset_120_real dataset_real
!mv dataset_120 masks_raw


In [54]:
import os
import cv2

mask_dir = "masks_raw/dataset_120"
img_dir = "dataset_real/dataset_120_real"
output_dir = "fixed_labels_yolo"
os.makedirs(output_dir, exist_ok=True)

# Obtener listas ordenadas de archivos (por nombre)
mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith(".png")])
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".png")])

# Verificar que haya la misma cantidad
assert len(mask_files) == len(img_files), "❌ Error: número de máscaras e imágenes no coincide"

for mask_file, img_file in zip(mask_files, img_files):
    mask_path = os.path.join(mask_dir, mask_file)
    img_path = os.path.join(img_dir, img_file)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.imread(img_path)

    if mask is None or img is None:
        print(f"⚠️ Problema leyendo {mask_file} o {img_file}")
        continue

    h, w = mask.shape[:2]

    # Encontrar contornos de la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear archivo de etiqueta YOLO
    label_file = os.path.join(output_dir, img_file.replace(".png", ".txt"))
    with open(label_file, "w") as f:
        for cnt in contours:
            x, y, bw, bh = cv2.boundingRect(cnt)
            xc = (x + bw / 2) / w
            yc = (y + bh / 2) / h
            bw /= w
            bh /= h
            f.write(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

print("✅ Archivos YOLO generados correctamente en:", output_dir)


✅ Archivos YOLO generados correctamente en: fixed_labels_yolo


In [55]:
from sklearn.model_selection import train_test_split
import shutil

# Crear carpetas destino
for split in ["train", "val"]:
    os.makedirs(f"dataset/images/{split}", exist_ok=True)
    os.makedirs(f"dataset/labels/{split}", exist_ok=True)

# Lista de imágenes y etiquetas
images = sorted([f for f in os.listdir("dataset_real/dataset_120_real") if f.endswith(".png")])
labels = sorted([f.replace(".png", ".txt") for f in images])

image_paths = [f"dataset_real/dataset_120_real/{img}" for img in images]
label_paths = [f"fixed_labels_yolo/{lbl}" for lbl in labels]

train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(image_paths, label_paths, test_size=0.2, random_state=42)

# Copiar imágenes y etiquetas
for split, imgs, lbls in [("train", train_imgs, train_lbls), ("val", val_imgs, val_lbls)]:
    for img_path, lbl_path in zip(imgs, lbls):
        shutil.copy(img_path, f"dataset/images/{split}/")
        shutil.copy(lbl_path, f"dataset/labels/{split}/")


In [56]:
%%writefile data.yaml
train: /content/dataset/images/train
val: /content/dataset/images/val
nc: 1
names: ['gota']


Writing data.yaml


In [57]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 17360 (delta 36), reused 18 (delta 18), pack-reused 17308 (from 2)
Receiving objects: 100% (17360/17360), 16.23 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (11898/11898), done.
/content/yolov5


In [58]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Opcional: evita conexión con Weights & Biases

!python train.py --img 640 --batch 16 --epochs 50 --data /content/data.yaml --weights yolov5s.pt --name gotas_yolo


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-31 18:08:39.247608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743444519.305606   31073 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743444519.322412   31073 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: unai-vazquez (unai-vazquez-esilv) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg

In [59]:
import shutil

# Comprimir los resultados del entrenamiento
shutil.make_archive("gotas_yolo_results", 'zip', "/content/yolov5/runs/train/gotas_yolo")


'/content/yolov5/gotas_yolo_results.zip'